In [50]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import os
import io

# API 설정
API_KEY = 's5DFRYyOQqKQxUWMjlKi9g'
AWS_URL = 'https://apihub.kma.go.kr/api/typ01/url/awsh.php'
ASOS_URL = 'https://apihub.kma.go.kr/api/typ01/url/kma_sfctm2.php'
SAVE_PATH = 'F:/박정현/ML/Mosquito/'

# DMS 시간 필터링: 전일 19시 ~ 당일 5시
def filter_time_range(df, start_time='19:00', end_time='05:00'):
    df.rename(columns={df.columns[0]: 'YYMMDDHHMI'}, inplace=True)
    df['시간'] = pd.to_datetime(df['YYMMDDHHMI'], format='%Y%m%d%H%M')
    df['시간'] = df['시간'].dt.strftime('%H:%M')  # 시간만 문자열로 추출

    return df[(df['시간'] >= start_time) | (df['시간'] <= end_time)]

# AWS 데이터 다운로드 함수
def download_aws_data(start_date, end_date):
    data_frames = []
    current_date = start_date

    while current_date <= end_date:
        params = {
            'authKey': API_KEY,
            'tm': current_date.strftime('%Y%m%d%H%M'),  # 시간 포함
            'var': '0',  # 모든 필요한 데이터 요청
            'stn': 0  # 전체 관측소 요청
        }

        response = requests.get(AWS_URL, params=params)
        if response.status_code == 200:
            raw_text = response.text
            raw_data = "\n".join(line for line in raw_text.splitlines() if not line.startswith("#"))
            df = pd.read_csv(io.StringIO(raw_data), sep='\s+')
            # 열 이름 확인 후 재설정
            if len(df.columns) == 10:
                df.columns = ['YYMMDDHHMI', 'STN', 'TA', 'WD', 'WS', 'RN_DAY', 'RN_HR1', 'HM', 'PA', 'PS']
            data_frames.append(df)
        else:
            print(f"Failed to fetch AWS data for {current_date}: {response.status_code}")

        current_date += timedelta(days=1)

    if data_frames:
        return pd.concat(data_frames, ignore_index=True)
    else:
        raise ValueError("No AWS data available for the given date range")

# ASOS 데이터 다운로드 함수
def download_asos_data(start_date, end_date):
    data_frames = []
    current_date = start_date

    while current_date <= end_date:
        params = {
            'authKey': API_KEY,
            'tm': current_date.strftime('%Y%m%d%H%M'),  # 시간 포함
            'stn': 0  # 전체 관측소 요청
        }

        response = requests.get(ASOS_URL, params=params)
        if response.status_code == 200:
            raw_text = response.text
            raw_data = "\n".join(line for line in raw_text.splitlines() if not line.startswith("#"))
            df = pd.read_csv(io.StringIO(raw_data), sep='\s+')
            # 열 이름 확인 후 재설정
            if len(df.columns) == 10:
                df.columns = ['YYMMDDHHMI', 'STN', 'TA', 'WD', 'WS', 'RN_DAY', 'RN_HR1', 'HM', 'PA', 'PS']
            data_frames.append(df)
        else:
            print(f"Failed to fetch ASOS data for {current_date}: {response.status_code}")

        current_date += timedelta(days=1)

    if data_frames:
        return pd.concat(data_frames, ignore_index=True)
    else:
        raise ValueError("No ASOS data available for the given date range")

# 일 단위 변환 함수
def aggregate_daily_data(df):
    df['날짜'] = pd.to_datetime(df['YYMMDDHHMI'], format='%Y%m%d%H%M')
    daily_data = df.groupby(df['날짜'].dt.date).agg({
        'TA': ['mean', 'max', 'min'],
        'RN_DAY': 'sum',
        'WS': 'mean',
        'HM': 'mean'
    }).reset_index()
    daily_data.columns = ['날짜', '기온_평균', '기온_최대', '기온_최소', '강수량_합', '풍속_평균', '습도_평균']
    return daily_data

# 결측치 처리 함수
def handle_missing_data(df, backup_station_data):
    # '날짜' 열이 있는지 확인
    if '날짜' not in df.columns:
        raise KeyError("'날짜' 열이 데이터프레임에 없습니다. 데이터 전처리 단계를 확인하세요.")
    
    # '날짜' 열이 datetime 형식인지 확인 및 변환
    if not pd.api.types.is_datetime64_any_dtype(df['날짜']):
        df['날짜'] = pd.to_datetime(df['날짜'], errors='coerce')

    # -99.0을 결측치로 간주
    df[['기온_평균', '기온_최대', '기온_최소', '강수량_합', '풍속_평균', '습도_평균']] = df[['기온_평균', '기온_최대', '기온_최소', '강수량_합', '풍속_평균', '습도_평균']].replace(-99.0, pd.NA)

    missing_dates = df[df.isnull().any(axis=1)]['날짜']

    for date in missing_dates:
        backup_data = backup_station_data[backup_station_data['날짜'] == date]
        if not backup_data.empty:
            for col in ['기온_평균', '기온_최대', '기온_최소', '강수량_합', '풍속_평균', '습도_평균']:
                df.loc[df['날짜'] == date, col] = backup_data[col].values[0]
        else:
            # 해당 날짜의 다른 연도 평균값으로 대체
            other_years_data = df[(df['날짜'].dt.month == date.month) & (df['날짜'].dt.day == date.day)]
            if not other_years_data.empty:
                for col in ['기온_평균', '기온_최대', '기온_최소', '강수량_합', '풍속_평균', '습도_평균']:
                    df.loc[df['날짜'] == date, col] = other_years_data[col].mean()

    return df.interpolate()

# 실행 예시
if __name__ == "__main__":
    start_date = datetime(2015, 3, 31)
    end_date = datetime(2015, 4, 5)

    try:
        aws_data = download_aws_data(start_date, end_date)
        asos_data = download_asos_data(start_date, end_date)

        filtered_aws_data = filter_time_range(aws_data)
        daily_aws_data = aggregate_daily_data(filtered_aws_data)

        # 결측치 보완
        processed_data = handle_missing_data(daily_aws_data, asos_data)

        # 결과 저장
        output_file = os.path.join(SAVE_PATH, "smapel2_weather_data.csv")
        processed_data.to_csv(output_file, index=False, encoding='utf-8-sig')
        print(f"File saved to: {os.path.abspath(output_file)}")
        print(f"Data saved to {output_file}")
    except ValueError as e:
        print(e)


KeyError: '날짜'

In [ ]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
import io

# API 설정
API_KEY = 's5DFRYyOQqKQxUWMjlKi9g'
AWS_URL = 'https://apihub.kma.go.kr/api/typ01/url/awsh.php'
ASOS_URL = 'https://apihub.kma.go.kr/api/typ01/url/kma_sfctm2.php'
SAVE_PATH = 'F:/박정현/ML/Mosquito/'

# DMS 시간 필터링: 전일 19시 ~ 당일 5시
def filter_time_range(df, start_time='19:00', end_time='05:00'):
    df.rename(columns={df.columns[0]: 'YYMMDDHHMI'}, inplace=True)
    df['시간'] = pd.to_datetime(df['YYMMDDHHMI'], format='%Y%m%d%H%M')
    df['시간'] = df['시간'].dt.strftime('%H:%M')  # 시간만 문자열로 추출

    return df[(df['시간'] >= start_time) | (df['시간'] <= end_time)]

# AWS 데이터 다운로드 함수
def download_aws_data(start_date, end_date):
    data_frames = []
    current_date = start_date

    while current_date <= end_date:
        params = {
            'authKey': API_KEY,
            'tm': current_date.strftime('%Y%m%d%H%M'),  # 시간 포함
            'var': '0',  # 모든 필요한 데이터 요청
            'stn': '0'  # 특정 관측소 요청
        }

        response = requests.get(AWS_URL, params=params)
        if response.status_code == 200:
            raw_text = response.text
            raw_data = response.text
            df = pd.read_csv(io.StringIO(raw_data), sep='\s+', comment='#', skip_blank_lines=True)
            # 열 이름 확인 후 재설정
            if len(df.columns) == 10:
                df.columns = ['YYMMDDHHMI', 'STN', 'TA', 'WD', 'WS', 'RN_DAY', 'RN_HR1', 'HM', 'PA', 'PS']
            data_frames.append(df)
        else:
            print(f"Failed to fetch AWS data for {current_date}: {response.status_code}")

        current_date += timedelta(days=1)

    if data_frames:
        return pd.concat(data_frames, ignore_index=True)
    else:
        raise ValueError("No AWS data available for the given date range")

# ASOS 데이터 다운로드 함수
def download_asos_data(start_date, end_date):
    data_frames = []
    current_date = start_date

    while current_date <= end_date:
        params = {
            'authKey': API_KEY,
            'tm': current_date.strftime('%Y%m%d%H%M'),  # 시간 포함
            'stn': 108  # 특정 관측소 요청
        }

        response = requests.get(ASOS_URL, params=params)
        if response.status_code == 200:
            raw_text = response.text
            raw_data = response.text
            df = pd.read_csv(io.StringIO(raw_data), sep='\s+', header=None, comment='#', skip_blank_lines=True)
            # 필요한 열만 선택
            if df.shape[1] >= 10:  # 최소 10개 이상의 열이 있을 경우
                df = df.iloc[:, [0, 1, 11, 3, 15, 13]]  # 필요한 열 선택: YYMMDDHHMI, STN, TA, WS, RN, HM
                df.columns = ['YYMMDDHHMI', 'STN', 'TA', 'WS', 'RN', 'HM']
                # '날짜' 열 추가
                df['날짜'] = pd.to_datetime(df['YYMMDDHHMI'], format='%Y%m%d%H%M').dt.date
            else:
                raise ValueError(f"Unexpected number of columns in ASOS data for {current_date}")
            data_frames.append(df)
        else:
            print(f"Failed to fetch ASOS data for {current_date}: {response.status_code}")

        current_date += timedelta(days=1)

    if data_frames:
        return pd.concat(data_frames, ignore_index=True)
    else:
        raise ValueError("No ASOS data available for the given date range")

# 일 단위 변환 함수
def aggregate_daily_data(df):
    df['날짜'] = pd.to_datetime(df['YYMMDDHHMI'], format='%Y%m%d%H%M')
    daily_data = df.groupby(['STN', df['날짜'].dt.date]).agg({
        'TA': ['mean', 'max', 'min'],
        'RN': 'sum',
        'WS': 'mean',
        'HM': 'mean'
    }).reset_index()
    daily_data.columns = ['지점', '날짜', '평균기온(℃)', '최고기온(℃)', '최저기온(℃)', '일강수량(mm)', '평균풍속(m/s)', '평균 상대습도(%)']
    return daily_data

# 결측치 처리 함수
def handle_missing_data(df, backup_station_data):
    # '날짜' 열이 datetime 형식인지 확인 및 변환
    if not pd.api.types.is_datetime64_any_dtype(df['날짜']):
        df['날짜'] = pd.to_datetime(df['날짜'], errors='coerce')

    # backup_station_data의 '날짜' 열도 datetime 형식으로 변환
    if '날짜' in backup_station_data.columns and not pd.api.types.is_datetime64_any_dtype(backup_station_data['날짜']):
        backup_station_data['날짜'] = pd.to_datetime(backup_station_data['날짜'], errors='coerce')
    
    # -99.0 및 -9.0을 결측치로 간주
    df[['평균기온(℃)', '최고기온(℃)', '최저기온(℃)', '일강수량(mm)', '평균풍속(m/s)', '평균 상대습도(%)']] = df[['평균기온(℃)', '최고기온(℃)', '최저기온(℃)', '일강수량(mm)', '평균풍속(m/s)', '평균 상대습도(%)']].replace([-99.0, -9.0], np.nan)

    missing_dates = df[df.isnull().any(axis=1)]['날짜']

    for date in missing_dates:
        backup_data = backup_station_data[backup_station_data['날짜'] == date]
        if not backup_data.empty:
            for col in ['평균기온(℃)', '최고기온(℃)', '최저기온(℃)', '일강수량(mm)', '평균풍속(m/s)', '평균 상대습도(%)']:
                df.loc[df['날짜'] == date, col] = backup_data[col].values[0]
        else:
            # 다른 연도의 같은 날짜 평균값으로 대체
            same_day_data = df[(df['날짜'].dt.month == date.month) & (df['날짜'].dt.day == date.day)]
            if not same_day_data.empty:
                for col in ['평균기온(℃)', '최고기온(℃)', '최저기온(℃)', '일강수량(mm)', '평균풍속(m/s)', '평균 상대습도(%)']:
                    df.loc[df['날짜'] == date, col] = same_day_data[col].mean()

    return df.interpolate()
# 실행 예시
if __name__ == "__main__":
    start_date = datetime(2015, 3, 31)
    end_date = datetime(2015, 4, 2)

    try:
        aws_data = download_aws_data(start_date, end_date)
        asos_data = download_asos_data(start_date, end_date)

        filtered_aws_data = filter_time_range(aws_data)
        daily_aws_data = aggregate_daily_data(filtered_aws_data)

        # ASOS 데이터도 일별 집계
        daily_asos_data = aggregate_daily_data(asos_data)

        # 데이터 확인 코드 추가
        print("Daily AWS Data Columns:", daily_aws_data.columns)
        print(daily_aws_data.head())
        print("Daily ASOS Data Columns:", daily_asos_data.columns)
        print(daily_asos_data.head())

        # 결측치 보완
        processed_data = handle_missing_data(daily_aws_data, daily_asos_data)

        # 결과 저장
        output_file = os.path.join(SAVE_PATH, "sample3_weather_data.csv")
        processed_data.to_csv(output_file, index=False, encoding='cp949')
        print(f"File saved to: {os.path.abspath(output_file)}")
        print(f"Data saved to {output_file}")
    except ValueError as e:
        print(e)


Daily AWS Data Columns: Index(['지점', '날짜', '평균기온(℃)', '최고기온(℃)', '최저기온(℃)', '일강수량(mm)', '평균풍속(m/s)',
       '평균 상대습도(%)'],
      dtype='object')
   지점          날짜  평균기온(℃)  최고기온(℃)  최저기온(℃)  일강수량(mm)  평균풍속(m/s)  평균 상대습도(%)
0   2  2015-03-31      7.5      7.5      7.5       0.0        1.1        56.6
1   2  2015-04-01     11.5     11.5     11.5       0.5        0.2        94.5
2   2  2015-04-02      3.8      3.8      3.8       0.0        0.9        83.4
3   3  2015-03-31     16.8     16.8     16.8       0.0        2.9         NaN
4   3  2015-04-01     17.0     17.0     17.0       0.0        0.4         NaN
File saved to: F:\박정현\ML\Mosquito\sample3_weather_data.csv
Data saved to F:/박정현/ML/Mosquito/sample3_weather_data.csv


In [98]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
import io

# API 설정
API_KEY = 's5DFRYyOQqKQxUWMjlKi9g'
AWS_URL = 'https://apihub.kma.go.kr/api/typ01/url/awsh.php'
ASOS_URL = 'https://apihub.kma.go.kr/api/typ01/url/kma_sfctm2.php'
SAVE_PATH = 'F:/박정현/ML/Mosquito/'

# DMS 시간 필터링: 전일 19시 ~ 당일 5시
def filter_time_range(df, start_time='19:00', end_time='05:00'):
    df.rename(columns={df.columns[0]: 'YYMMDDHHMI'}, inplace=True)
    df['시간'] = pd.to_datetime(df['YYMMDDHHMI'], format='%Y%m%d%H%M')
    df['시간'] = df['시간'].dt.strftime('%H:%M')  # 시간만 문자열로 추출

    return df[(df['시간'] >= start_time) | (df['시간'] <= end_time)]

# AWS 데이터 다운로드 함수
def download_aws_data(start_date, end_date):
    data_frames = []
    current_date = start_date

    while current_date <= end_date:
        params = {
            'authKey': API_KEY,
            'tm': current_date.strftime('%Y%m%d%H%M'),  # 시간 포함
            'var': '0',  # 모든 필요한 데이터 요청
            'stn': '0'  # 특정 관측소 요청
        }

        response = requests.get(AWS_URL, params=params)
        if response.status_code == 200:
            raw_text = response.text
            raw_data = response.text
            df = pd.read_csv(io.StringIO(raw_data), sep='\s+', comment='#', skip_blank_lines=True)
            # 열 이름 확인 후 재설정
            if len(df.columns) == 10:
                df.columns = ['YYMMDDHHMI', 'STN', 'TA', 'WD', 'WS', 'RN_DAY', 'RN_HR1', 'HM', 'PA', 'PS']
            data_frames.append(df)

            for df in data_frames:
                df[['YYMMDDHHMI', 'STN', 'TA', 'WD', 'WS', 'RN_DAY', 'RN_HR1', 'HM', 'PA', 'PS']] = df[['YYMMDDHHMI', 'STN', 'TA', 'WD', 'WS', 'RN_DAY', 'RN_HR1', 'HM', 'PA', 'PS']].replace(-99, np.nan)
        else:
            print(f"Failed to fetch AWS data for {current_date}: {response.status_code}")

        current_date += timedelta(days=1)

    if data_frames:
        return pd.concat(data_frames, ignore_index=True)
    else:
        raise ValueError("No AWS data available for the given date range")

# ASOS 데이터 다운로드 함수
# 
    data_frames = []
    current_date = start_date

    while current_date <= end_date:
        params = {
            'authKey': API_KEY,
            'tm': current_date.strftime('%Y%m%d%H%M'),  # 시간 포함
            'stn': 108  # 특정 관측소 요청
        }

        response = requests.get(ASOS_URL, params=params)
        if response.status_code == 200:
            raw_text = response.text
            raw_data = response.text
            df = pd.read_csv(io.StringIO(raw_data), sep='\s+', header=None, comment='#', skip_blank_lines=True)
            # 필요한 열만 선택
            if df.shape[1] >= 10:  # 최소 10개 이상의 열이 있을 경우
                df = df.iloc[:, [0, 1, 11, 3, 15, 13]]  # 필요한 열 선택: YYMMDDHHMI, STN, TA, WS, RN, HM
                df.columns = ['YYMMDDHHMI', 'STN', 'TA', 'WS', 'RN', 'HM']
                # '날짜' 열 추가
                df['날짜'] = pd.to_datetime(df['YYMMDDHHMI'], format='%Y%m%d%H%M').dt.date
            else:
                raise ValueError(f"Unexpected number of columns in ASOS data for {current_date}")
            data_frames.append(df)
        else:
            print(f"Failed to fetch ASOS data for {current_date}: {response.status_code}")

        current_date += timedelta(days=1)

    if data_frames:
        return pd.concat(data_frames, ignore_index=True)
    else:
        raise ValueError("No ASOS data available for the given date range")

# 일 단위 변환 함수
def aggregate_daily_data(df):
    df['날짜'] = pd.to_datetime(df['YYMMDDHHMI'], format='%Y%m%d%H%M')
    daily_data = df.groupby(['STN', df['날짜'].dt.date]).agg({
        'TA': ['mean', 'max', 'min'],
        'RN_HR1': 'sum',
        'WS': 'mean',
        'HM': 'mean'
    }).reset_index()
    daily_data.columns = ['지점', '날짜', '평균기온(℃)', '최고기온(℃)', '최저기온(℃)', '일강수량(mm)', '평균풍속(m/s)', '평균 상대습도(%)']
    return daily_data

# 결측치 처리 함수
def handle_missing_data(df, backup_station_data):
    # '날짜' 열이 datetime 형식인지 확인 및 변환
    if not pd.api.types.is_datetime64_any_dtype(df['날짜']):
        df['날짜'] = pd.to_datetime(df['날짜'], errors='coerce')

    # backup_station_data의 '날짜' 열도 datetime 형식으로 변환
    if '날짜' in backup_station_data.columns and not pd.api.types.is_datetime64_any_dtype(backup_station_data['날짜']):
        backup_station_data['날짜'] = pd.to_datetime(backup_station_data['날짜'], errors='coerce')
    
    # -99.0 및 -9.0을 결측치로 간주
    df[['평균기온(℃)', '최고기온(℃)', '최저기온(℃)', '일강수량(mm)', '평균풍속(m/s)', '평균 상대습도(%)']] = df[['평균기온(℃)', '최고기온(℃)', '최저기온(℃)', '일강수량(mm)', '평균풍속(m/s)', '평균 상대습도(%)']].replace(-99, np.nan)

    missing_dates = df[df.isnull().any(axis=1)]['날짜']

    for date in missing_dates:
        backup_data = backup_station_data[backup_station_data['날짜'] == date]
        if not backup_data.empty:
            for col in ['평균기온(℃)', '최고기온(℃)', '최저기온(℃)', '일강수량(mm)', '평균풍속(m/s)', '평균 상대습도(%)']:
                df.loc[df['날짜'] == date, col] = backup_data[col].values[0]
        else:
            # 다른 연도의 같은 날짜 평균값으로 대체
            same_day_data = df[(df['날짜'].dt.month == date.month) & (df['날짜'].dt.day == date.day)]
            if not same_day_data.empty:
                for col in ['평균기온(℃)', '최고기온(℃)', '최저기온(℃)', '일강수량(mm)', '평균풍속(m/s)', '평균 상대습도(%)']:
                    df.loc[df['날짜'] == date, col] = same_day_data[col].mean()

    return df.interpolate()
# 실행 예시
if __name__ == "__main__":
    start_date = datetime(2015, 3, 31)
    end_date = datetime(2015, 4, 2)

    aws_data = download_aws_data(start_date, end_date)

    filtered_aws_data = filter_time_range(aws_data)
    daily_aws_data = aggregate_daily_data(filtered_aws_data)

    # 데이터 확인 코드 추가
    print("Daily AWS Data Columns:", daily_aws_data.columns)
    print(daily_aws_data.head())
        

    # 결과 저장
    output_file = os.path.join(SAVE_PATH, "sample3_weather_data.csv")
    daily_aws_data.to_csv(output_file, index=False, encoding='cp949')
    print(f"File saved to: {os.path.abspath(output_file)}")
    print(f"Data saved to {output_file}")
        




Daily AWS Data Columns: Index(['지점', '날짜', '평균기온(℃)', '최고기온(℃)', '최저기온(℃)', '일강수량(mm)', '평균풍속(m/s)',
       '평균 상대습도(%)'],
      dtype='object')
   지점          날짜  평균기온(℃)  최고기온(℃)  최저기온(℃)  일강수량(mm)  평균풍속(m/s)  평균 상대습도(%)
0   2  2015-03-31      7.5      7.5      7.5       0.0        1.1        56.6
1   2  2015-04-01     11.5     11.5     11.5       0.5        0.2        94.5
2   2  2015-04-02      3.8      3.8      3.8       0.0        0.9        83.4
3   3  2015-03-31     16.8     16.8     16.8       0.0        2.9         NaN
4   3  2015-04-01     17.0     17.0     17.0       0.0        0.4         NaN
File saved to: F:\박정현\ML\Mosquito\sample3_weather_data.csv
Data saved to F:/박정현/ML/Mosquito/sample3_weather_data.csv


In [97]:
%reset -f